<a href="https://colab.research.google.com/github/raiadi96/Pytorch/blob/master/StackOverFlowTagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import os
import shutil
import numpy as np
import string
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing

In [14]:
#fetch the dataset
!tar -xzf stack_overflow_16k.tar.gz

tar (child): stack_overflow_16k.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [15]:
os.listdir()

['.config', 'README.md', 'test', 'train', '.ipynb_checkpoints', 'sample_data']

In [16]:
train_dir = os.path.abspath('train')
os.listdir(train_dir)

['javascript', 'csharp', 'java', 'python']

In [17]:
sample_file = os.path.join(train_dir, 'python/1991.txt')
with open(sample_file) as f:
  print(f.read())

"how to automatically add unique key according to user input and then update it in blank i have a problem creating a dictionary key that is unique and comes from the user. and i don't know how to add it. so here is what i desire:..sale: blue 5.5 blue so far.sale: orange 2.2 orange so far.sale: blue 3.8 blue so far.sale: ...how do i update it?..object1 = {}..line = input('sales: ').while line:.  parts = line.split().  key = parts[0].  val = (parts[1])*1.  object1[key] = int(val).  print(object1).  line = input('sales: ')...firstly, i want to know how to add unique key to dictionary and second how do i add it."



In [19]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'train',
    batch_size = batch_size,
    validation_split = 0.2,
    subset = 'training',
    seed = seed
)


Found 8000 files belonging to 4 classes.
Using 6400 files for training.


In [22]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(1):
    print("Question : {0}".format(text_batch.numpy()[i]))
    print("Label : {0}".format(label_batch.numpy()[i]))
    print("{0} is equivalent to {1}".format(label_batch.numpy()[i], raw_train_ds.class_names[label_batch.numpy()[i]]))


Question : b'"unit testing of setters and getters teacher wanted us to do a comprehensive unit test. for me, this will be the first time that i use junit. i am confused about testing set and get methods. do you think should i test them? if the answer is yes; is this code enough for testing?..  public void testsetandget(){.    int a = 10;.    class firstclass = new class();.    firstclass.setvalue(10);.    int value = firstclass.getvalue();.    assert.asserttrue(""error"", value==a);.  }...in my code, i think if there is an error, we can\'t know that the error is deriving because of setter or getter."\n'
Label : 1
1 is equivalent to java


In [24]:
#similarly let us create validation and test dataset

raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'train',
    validation_split = 0.2,
    batch_size = batch_size,
    subset = 'validation',
    seed = seed
)

Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [25]:
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'test',
    batch_size = batch_size
)

Found 8000 files belonging to 4 classes.


In [40]:
#standardization of text data
import re
def custom_standarization(text_data):
  lowercase = tf.strings.lower(text_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />',' ')
  return tf.strings.regex_replace(stripped_html,re.escape(string.punctuation), '')


In [41]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

max_features = 10000
sequence_length = 250

vectorize_layer = TextVectorization(
    standardize = custom_standarization,
    max_tokens = max_features,
    output_mode = 'int',
    output_sequence_length = sequence_length
)

In [42]:
train_text = raw_train_ds.map(lambda x,y : x)
vectorize_layer.adapt(train_text)

In [43]:
def vectorize_text(text, label):
  text = tf.expand_dims(text,-1)
  return vectorize_layer(text), label

In [45]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [47]:
#creating a cache inorder to make sure data processing does not become a bottle neck while running a model
AUTOTUNE = tf.data.experimental.AUTOTUNE 

train_ds = train_ds.cache().prefetch(buffer_size = AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size = AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size = AUTOTUNE)

In [55]:
embedding_dim = 16

model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(4)])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          160016    
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 16)          0         
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 68        
Total params: 160,084
Trainable params: 160,084
Non-trainable params: 0
_________________________________________________________________


In [56]:
model.compile(loss=losses.SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=[
  'accuracy'
])


In [57]:

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5)

Epoch 1/5
200/200 [==============================] - 4s 18ms/step - loss: 1.3789 - accuracy: 0.3461 - val_loss: 1.3676 - val_accuracy: 0.4050
Epoch 2/5
200/200 [==============================] - 1s 7ms/step - loss: 1.3502 - accuracy: 0.4769 - val_loss: 1.3284 - val_accuracy: 0.5238
Epoch 3/5
200/200 [==============================] - 1s 7ms/step - loss: 1.2983 - accuracy: 0.5342 - val_loss: 1.2681 - val_accuracy: 0.5619
Epoch 4/5
200/200 [==============================] - 1s 7ms/step - loss: 1.2286 - accuracy: 0.5797 - val_loss: 1.1957 - val_accuracy: 0.6175
Epoch 5/5
200/200 [==============================] - 1s 7ms/step - loss: 1.1513 - accuracy: 0.6311 - val_loss: 1.1208 - val_accuracy: 0.6694


In [58]:

loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

250/250 [==============================] - 3s 12ms/step - loss: 1.1331 - accuracy: 0.6505
Loss:  1.1330634355545044
Accuracy:  0.6504999995231628
